# 校正Syntatic Structure（耗时约xx小时）

## 4.1 校正Symmetry和Asymmetry（包含指定词性和句式的句子对）

In [ ]:
#对比是否是对调指定字左右两边的字词
def verify_word_switch(text_1, text_2, word):
    # word左右字符数量初始化
    right_length = 0
    left_length = 0
    # 对于text_1中的每一个字符
    for i in range(0, len(text_1)):
        # 除指定word外，从左往右，若在同一个位置找到相同的字符则替换为“ ”
        if text_1[i] != word and text_1[i] == text_2[i]:
            text_1 = text_1[:i] + " " + text_1[i+1:]
            text_2 = text_2[:i] + " " + text_2[i+1:]
    # 将“ ” 替换为“” 
    text_1 = text_1.replace(" ","")
    text_2 = text_2.replace(" ","")
    # print(text_1, text_2)
    # 如果剩下的字词长度一致
    if len(text_1) == len(text_2):
        # 获得两个句子中word存在的数量
        count_word_1 = text_1.count(word)
        count_word_2 = text_2.count(word)
        # 若句子中word数量相同
        if count_word_1 == count_word_2:
            # 仅存在一个word时
            if  count_word_1 == 1:
                word_index_1 = text_1.index(word)
                word_index_2 = text_2.index(word)
                return check_left_right(text_1, text_2, word_index_1, word_index_2)
            # 存在多个word时
            elif count_word_1 == 2:
                w_1_index_list = []
                w_2_index_list = []
                for w_1 in range(0, len(text_1)):
                    if text_1[w_1] == word: w_1_index_list.append(w_1)
                for w_2 in range(0, len(text_2)):
                    if text_2[w_2] == word: w_2_index_list.append(w_2)
                condition_1 = check_left_right(text_1, text_2, w_1_index_list[0], w_2_index_list[1])
                condition_2 = check_left_right(text_1, text_2, w_1_index_list[1], w_2_index_list[0])
                if condition_1 or condition_2:
                    return True
                else: 
                    return False

# 检查指定               
def check_left_right(text_1, text_2, word_index_1, word_index_2):
    num_word_removed = 0
    # print(word_index_1, word_index_2)
    # 找出word左边字符的最小数量
    if word_index_1 <= word_index_2:
        left_length = word_index_1
    else:
        left_length = word_index_2
    # 找出word右边字符的最小数量   
    if len(text_1) - word_index_1 <= len(text_2) - word_index_2:
        right_length = len(text_1) - word_index_1
    else:
        right_length = len(text_2) - word_index_2
    # print(right_length, left_length)
    # 如果word在句子头或尾则无内容对调直接返回False
    if right_length == 0 or left_length == 0:
        return False
    else:
        # 在word左边，离word相同距离若有相同的字则替换为" "
        for j in range (1, left_length):
            x = word_index_1 - j
            y = word_index_2 - j
            if text_1[x] == text_2[y]:
                num_word_removed = num_word_removed + 1
                text_1 = text_1[:x] + " " + text_1[x+1:]
                text_2 = text_2[:y] + " " + text_2[y+1:]
         # 在word右边，离word相同距离若有相同的字则替换为" "
        for k in range (1, right_length):
            x = word_index_1 + k
            y = word_index_2 + k
            if text_1[x] == text_2[y]:
                text_1 = text_1[:x] + " " + text_1[x+1:]
                text_2 = text_2[:y] + " " + text_2[y+1:] 
    # print(text_1, text_2)
    # 将“ ” 替换为“” 
    text_1 = text_1.replace(" ","")
    text_2 = text_2.replace(" ","")
    # print(text_1, text_2)
    # 以word为中心，切开两边
    word_index_1 = word_index_1 - num_word_removed
    word_index_2 = word_index_2 - num_word_removed
    a1 = text_1[0:word_index_1]
    a2 = text_1[word_index_1+1:len(text_1)]
    b1 = text_2[0:word_index_2]
    b2 = text_2[word_index_2+1:len(text_2)]
    # print(a1, a2)
    # print(b1, b2)
    # 如果 是switch 返回True
    if (a1 == b2 and a2 == b1):
        return True
    # 如果 不是 返回Faslse
    else:
        return False

In [ ]:
# 方法 3-3-1 寻找包含指定 字 的句子对
def get_symmetry_asymmetry_by_word(words_list, data_list_1, data_list_2, data_label_list = None, exclusion_words_list = [], print_progress = False):
    # 不再处理mispelling处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    text_pairs_list_a = []
    label_list_a = []
    text_pairs_list_b = []
    label_list_b = []
    print("="*15 + " In Progress " + "="*15)
    count = 0
    # 对每一个句子对进行分析
    for i in range (0, len(data_list_1)):
        # 去除句子对的标点符号
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]

        # 判断句子一里的所有字都在句子二中或者反之
        all_words_of_text_1_in_text_2 = True
        all_words_of_text_2_in_text_1 = True
        
        if (len(text_1) == len(text_2)) and ((text_1 + "\t" + text_2) not in misspelling_list):
            for x in text_1:
                if x not in text_2:
                    all_words_of_text_1_in_text_2 = False
            for y in text_2:
                if y not in text_1:
                    all_words_of_text_1_in_text_2 = False
            # 若 句子1 不等于 句子2，且 句子1或句子2的所有字都在另一个句子中
            if text_1 == text_2 and (all_words_of_text_1_in_text_2 or all_words_of_text_1_in_text_2):
                for word in words_list:
                    if word in text_1 and word in text_2:
                        if exclusion_words_list == []:
                            text_pairs_list_a.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                label_list_a.append(str(i) + "\t" + data_label_list[i])  
                        else:
                            text_pairs_list_b.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                label_list_b.append(str(i) + "\t" + data_label_list[i])  
            elif text_1 != text_2 and (all_words_of_text_1_in_text_2 or all_words_of_text_1_in_text_2):
                #对于每一个指定 词
                for word in words_list:
                    # 如果同时存在于两个句子中, 且是对调指定字左右两边的字词
                    if word in text_1 and word in text_2 and verify_word_switch(text_1, text_2, word):
                        if exclusion_words_list == []:
                            count = count + 1
                            text_pairs_list_a.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                label_list_a.append(str(i) + "\t" + data_label_list[i])  
                            break
                        else:
                            is_contain_exclusion = False
                            for exclusion_word in exclusion_words_list:
                                if exclusion_word in text_1 and exclusion_word in text_2:
                                    text_pairs_list_b.append(data_list_1[i] + "\t" + data_list_2[i] )
                                    if data_label_list!= None:
                                        label_list_b.append(str(i) + "\t" + data_label_list[i])  
                                    is_contain_exclusion = True
                                    break
                            if not is_contain_exclusion:
                                count = count + 1
                                text_pairs_list_a.append(data_list_1[i] + "\t" + data_list_2[i] )
                                if data_label_list!= None:
                                    label_list_a.append(str(i) + "\t" + data_label_list[i])  
                                break   
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for symmetry/asymmetry identification have processed " %((i/len(data_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    return text_pairs_list_a, label_list_a, text_pairs_list_b, label_list_b

In [ ]:
# 寻找symmetry的句子对
# 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
# symmetry
words_list = ["和", "同", "及", "或"] # "跟","与"
exclusion_words_list = []

text_pairs_1_list, label_1_list, text_pairs_0_list, label_0_list = get_symmetry_asymmetry_by_word( words_list, test_text_pair_1_list,test_text_pair_2_list, view_label_list, exclusion_words_list, True)
write_file("common_data/symmetry_1_by_word.txt",text_pairs_1_list)

# 打印结果
print_dataset(text_pairs_1_list, label_1_list,200)
print_dataset(text_pairs_0_list, label_0_list,200)



The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for symmetry/asymmetry identification have processed 
10.00 of text pairs for symmetry/asymmetry identification have processed 
20.00 of text pairs for symmetry/asymmetry identification have processed 
30.00 of text pairs for symmetry/asymmetry identification have processed 
40.00 of text pairs for symmetry/asymmetry identification have processed 
50.00 of text pairs for symmetry/asymmetry identification have processed 
60.00 of text pairs for symmetry/asymmetry identification have processed 
70.00 of text pairs for symmetry/asymmetry identification have processed 
80.00 of text pairs for symmetry/asymmetry identification have processed 
90.00 of text pairs for symmetry/asymmetry identification have processed 
===============     Done    ===============
===============  

In [ ]:
# 寻找asymmetry的句子对 (按照 字)
# 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
# asymmetry 飞/到/去/至
words_list = ['飞', '到', '去', '至', "离", "距"]
exclusion_words_list = ['多少', '高速费', '邮费', '多远','多少公里','距离', '多久','多长时间']
text_pairs_0_list, label_0_list, text_pairs_1_list, label_1_list = get_symmetry_asymmetry_by_word( words_list, test_text_pair_1_list,test_text_pair_2_list, view_label_list, exclusion_words_list, True)
print_dataset(text_pairs_1_list, label_1_list,200)
print_dataset(text_pairs_0_list, label_0_list,200)

write_file("common_data/asymmetry_1_by_word.txt",text_pairs_1_list )
write_file("common_data/asymmetry_0_by_word.txt",text_pairs_0_list )

# 打印结果
print("asymmetry_1_list: " + str(len(text_pairs_1_list)) + " 组")
print("asymmetry_0_list: " + str(len(text_pairs_0_list)) + " 组")



The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for symmetry/asymmetry identification have processed 
10.00 of text pairs for symmetry/asymmetry identification have processed 
20.00 of text pairs for symmetry/asymmetry identification have processed 
30.00 of text pairs for symmetry/asymmetry identification have processed 
40.00 of text pairs for symmetry/asymmetry identification have processed 
50.00 of text pairs for symmetry/asymmetry identification have processed 
60.00 of text pairs for symmetry/asymmetry identification have processed 
70.00 of text pairs for symmetry/asymmetry identification have processed 
80.00 of text pairs for symmetry/asymmetry identification have processed 
90.00 of text pairs for symmetry/asymmetry identification have processed 
===============     Done    ===============
===============  

## 4.2 校正Negative Asymmetry （包含指定词性和句式但仅差别反义词）

In [ ]:
# 去除指定word左右相同的字               
def remove_left_right(text_1, text_2, word_index_1, word_index_2, word):
    antonym_list = get_antonym_list()
    num_word_removed = 0
    # print(word_index_1, word_index_2)
    # 找出word左边字符的最小数量
    if word_index_1 <= word_index_2:
        left_length = word_index_1
    else:
        left_length = word_index_2
    # 找出word右边字符的最小数量   
    if len(text_1) - word_index_1 <= len(text_2) - word_index_2:
        right_length = len(text_1) - word_index_1
    else:
        right_length = len(text_2) - word_index_2
    # print(right_length, left_length)
    # 在word左边，离word相同距离若有相同的字则替换为" "
    for j in range (0, left_length):
        x = word_index_1 - j - 1
        y = word_index_2 - j - 1
        if text_1[x] != word and text_1[x] == text_2[y]:
            num_word_removed = num_word_removed + 1
            text_1 = text_1[:x] + " " + text_1[x+1:]
            text_2 = text_2[:y] + " " + text_2[y+1:]
    # 在word右边，离word相同距离若有相同的字则替换为" "
    for k in range (0, right_length):
        x = word_index_1 + k
        y = word_index_2 + k
        if text_1[x] != word and text_1[x] == text_2[y]:
            text_1 = text_1[:x] + " " + text_1[x+1:]
            text_2 = text_2[:y] + " " + text_2[y+1:] 
    # print(text_1, text_2)
    # 将“ ” 替换为“” 
    text_1 = text_1.replace(" ","")
    text_2 = text_2.replace(" ","")
    # print(text_1, text_2)
    # 如果去除后word成为了句子开头和结尾的字，则不是目标句
    if text_1[0] == word or text_1[-1] == word or text_2[0] == word or text_2[-1] == word:
        return "It is not our target"
    else:
        # 重新定位word
        word_index_1 = word_index_1 - num_word_removed
        word_index_2 = word_index_2 - num_word_removed
        # 以word为中心切开两边
        a1 = text_1[0:word_index_1]
        a2 = text_1[word_index_1+1:len(text_1)]
        b1 = text_2[0:word_index_2]
        b2 = text_2[word_index_2+1:len(text_2)]

        # print(a1, a2)
        # print(b1, b2)
        # 如果 如果是 A比Bxxx vs B比Axxx，则为0
        if (a1 == b2 and a2 == b1):
            # print("a1==b2 or a2 == b1" + text_1 + "\t" + text_2)
            return "0"
        # 如果 不是 
        else:
            is_negative_asymmetry = False
            # 如果 a句前部被包含在b句后部，b句前部被包含在a句后部
            is_a_in_b = False
            is_b_in_a = False
            for w_a in a1:
                if w_a in b2:
                    is_a_in_b = True
            for w_b in b1:
                if w_b in a2:
                    is_b_in_a = True

            if is_a_in_b and is_b_in_a:
                # print("a1 in b2 or b1 in a2" + text_1 + "\t" + text_2)
                # 判断两句的后部是否包含成对的反义词
                for antonym in antonym_list:
                    # 如果 列表中的词 出现在前后两部分说明并非是比较词
                    if ((antonym[0] in a1) and (antonym[0] in a2)) or ((antonym[1] in a1) and (antonym[1] in a2)) or ((antonym[0] in b1) and (antonym[0] in b2)) or ((antonym[1] in b1) and (antonym[1] in b2)):
                        abc =1
                    # 如果 是 作为比较词 且是反义词
                    elif (antonym[0] in a2 and antonym[1] in b2) or (antonym[1] in a2 and antonym[0] in b2):
                        # print("1")
                        # print(antonym)
                        is_negative_asymmetry = True
                        return "1"
                    # 如果 是 作为比较词 但相同
                    elif (antonym[0] in a2 and antonym[0] in b2) or (antonym[1] in a2 and antonym[1] in b2):
                        # print("0")
                        # print(antonym)
                        return "0"
            # print(a1, a2)
            # print(b1, b2)
            return "unknown"

# 方法 3-4 探索NA句型
def get_negative_asymmetry_by_word(words_list, data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    special_words_list = ["比例"]
    na_0_text_pairs_list = []
    na_1_text_pairs_list = []
    na_0_label_list = []
    na_1_label_list = []
    print("="*15 + " In Progress " + "="*15)
    count = 0
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]

        is_contain_special_words = False
        for special_word in special_words_list:
            if special_word in text_1 and special_word in text_2:
                is_contain_special_words = True

        for word in words_list:
            if text_1 != text_2 and (word in text_1 or word in text_2) and ((text_1 + "\t" + text_2) not in misspelling_list):
                # 情况一，带比仅仅差别一个反义词，该情况为0:
                subset_1, subset_2 = check_subset(text_1, text_2)
                if (not is_contain_special_words) and check_antonym(subset_1,subset_2):
                    # print("仅差别一个反义词， 0: " + text_1 + " \t " + text_2 )
                    na_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                    if data_label_list!= None:
                        na_0_label_list.append(str(i) + "\t" + data_label_list[i])  
                    break

                # 获得两个句子中word存在的数量
                count_word_1 = text_1.count(word)
                count_word_2 = text_2.count(word)
                # 若句子中word数量相同
                if (not is_contain_special_words) and count_word_1 == count_word_2:
                    # 仅存在一个word时, 且word不在句子开头也不在结尾
                    if  count_word_1 == 1 and text_1[0] != word and text_1[-1]!= word and text_2[0] != word and text_2[-1]!= word:
                        word_index_1 = text_1.index(word)
                        word_index_2 = text_2.index(word)
                        # 以word为中心，切开两边，去除相同的字
                        condition = remove_left_right(text_1, text_2, word_index_1, word_index_2, word)
                        if condition == "1":
                            # print("1: " + text_1 + " \t " + text_2 )
                            na_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                na_1_label_list.append(str(i) + "\t" + data_label_list[i])  
                            break
                        elif condition == "0":
                            # print("0: " + text_1 + " \t " + text_2 )
                            na_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                na_0_label_list.append(str(i) + "\t" + data_label_list[i]) 
                            break 
                        elif condition == "unknown":
                            # print("unknown_by_word: " + text_1 + " \t " + text_2 )
                            # 是否需要把无法分辨的按照原有标签存入？
                            # if data_label_list[i] == "0":
                            #     na_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            #     if data_label_list!= None:
                            #         na_0_label_list.append(str(i) + "\t" + data_label_list[i])
                            # elif data_label_list[i] == "1":
                            #     na_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            #     if data_label_list!= None:
                            #         na_1_label_list.append(str(i) + "\t" + data_label_list[i]) 
                            break
                # 对剩下的取出带有['比', 'p']的句子对
                words_1, natures_1 = get_words_and_words_nature(text_1)
                words_2, natures_2 = get_words_and_words_nature(text_2)
                words_natures_1 = merge_word_nature(words_1,natures_1)
                words_natures_2 = merge_word_nature(words_2,natures_2)
                if ['比', 'p'] in words_natures_1 or ['比', 'p'] in words_natures_2:
                    abc = 1
                    # print("unknown_by_nature: " + text_1 + " \t " + text_2 )
                    # 是否需要把无法分辨的按照原有标签存入？
                    # if data_label_list[i] == "0":
                    #     na_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                    #     if data_label_list!= None:
                    #         na_0_label_list.append(str(i) + "\t" + data_label_list[i])
                    # elif data_label_list[i] == "1":
                    #     na_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                    #     if data_label_list!= None:
                    #         na_1_label_list.append(str(i) + "\t" + data_label_list[i]) 
        
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for negative asymmetry identification have processed " %((i/len(data_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    print("带" + str(words_list) + "符号的有: " + str(count) + " 对\n")
    return  na_1_text_pairs_list, na_1_label_list, na_0_text_pairs_list, na_0_label_list


In [ ]:
# 寻找Negative Asymmetry的句子对
# 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
# symmetry
words_list = ['比']

na_1_text_pairs_list, na_1_label_list, na_0_text_pairs_list, na_0_label_list = get_negative_asymmetry_by_word(words_list, test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/negative_asymmetry_1_by_word.txt",na_1_text_pairs_list)
write_file("common_data/negative_asymmetry_0_by_word.txt",na_0_text_pairs_list)

# 打印结果
print_dataset(na_1_text_pairs_list, na_1_label_list,200)
print_dataset(na_0_text_pairs_list, na_0_label_list,200)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for negative asymmetry identification have processed 
10.00 of text pairs for negative asymmetry identification have processed 
20.00 of text pairs for negative asymmetry identification have processed 
30.00 of text pairs for negative asymmetry identification have processed 
40.00 of text pairs for negative asymmetry identification have processed 
50.00 of text pairs for negative asymmetry identification have processed 
60.00 of text pairs for negative asymmetry identification have processed 
70.00 of text pairs for negative asymmetry identification have processed 
80.00 of text pairs for negative asymmetry identification have processed 
90.00 of text pairs for negative asymmetry identification have processed 
===============     Done    ===============
带['比']符号的有: 0 对



## 4.3 寻找不同词为反义词的句子对

In [ ]:
#  探索带有反义词的句子对
def get_antonym_text_pairs(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    text_pairs_list = []
    label_list = []
    print("="*15 + " In Progress " + "="*15)
    count = 0
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]

        subset_1, subset_2 = check_subset(text_1, text_2)

        if check_antonym(subset_1,subset_2) and ((text_1 + "\t" + text_2) not in misspelling_list):
            # print("仅差别一个反义词， 0: " + text_1 + " \t " + text_2 )
            text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
            if data_label_list!= None:
                label_list.append(str(i) + "\t" + data_label_list[i])   
        
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for negative asymmetry identification have processed " %((i/len(data_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    print("带" + str(words_list) + "符号的有: " + str(count) + " 对\n")
    return text_pairs_list, label_list

# 寻找Negative Asymmetry的句子对
# 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
text_pairs_list, label_list = get_antonym_text_pairs(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/antonym_0_text_pairs.txt",text_pairs_list)

# 打印结果
print_dataset(text_pairs_list, label_list,200)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for negative asymmetry identification have processed 
10.00 of text pairs for negative asymmetry identification have processed 
20.00 of text pairs for negative asymmetry identification have processed 
30.00 of text pairs for negative asymmetry identification have processed 
40.00 of text pairs for negative asymmetry identification have processed 
50.00 of text pairs for negative asymmetry identification have processed 
60.00 of text pairs for negative asymmetry identification have processed 
70.00 of text pairs for negative asymmetry identification have processed 
80.00 of text pairs for negative asymmetry identification have processed 
90.00 of text pairs for negative asymmetry identification have processed 
===============     Done    ===============
带['比']符号的有: 0 对



## 4.4 带加减乘除的句子对处理

In [ ]:
# 检查指定符号前后数字是否是对调         
def check_time_add(text_1, text_2, word):
    word_index_1 = text_1.index(word)
    word_index_2 = text_2.index(word)
    # 找出word左边字符的最小数量
    if word_index_1 <= word_index_2:
        left_length = word_index_1
    else:
        left_length = word_index_2
    # 找出word右边字符的最小数量   
    if len(text_1) - word_index_1 <= len(text_2) - word_index_2:
        right_length = len(text_1) - word_index_1
    else:
        right_length = len(text_2) - word_index_2
    # print(right_length, left_length)
    # 如果word在句子头或尾则无内容对调直接返回False
    if right_length == 0 or left_length == 0:
        return False
    else:
        # print(text_1, text_2)
        # 以word为中心，切开两边
        word_index_1 = word_index_1 
        word_index_2 = word_index_2 
        a1 = text_1[0:word_index_1]
        a2 = text_1[word_index_1+1:len(text_1)]
        b1 = text_2[0:word_index_2]
        b2 = text_2[word_index_2+1:len(text_2)]
        # print(a1, a2)
        # print(b1, b2)
        # 如果 是switch 返回True
        if (a1 == b2 and a2 == b1):
            return True
        # 如果 不是 返回Faslse
        else:
            return False


In [ ]:
# 寻找带有加减乘除的的句子对
import re
# 方法3-5 抽取包含指定字或字符的句子对
def get_textpairs_with_calculations(data_list_1, data_list_2, data_label_list = None, print_progress = False):    
    # 不再处理mispelling处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    cal_0_text_pairs_list = []
    cal_0_label_list = []
    cal_1_text_pairs_list = []
    cal_1_label_list = []
    count = 0
    if print_progress: print("\n" + "="*15 + " In Progress " + "="*15)
    cal_words = [["加上","加","+"],["减去","减","-"],["乘以","乘","×"], ["乘以","乘","*"], ["除以","除","÷"], ["除以","除","/"]]
    exclusion_words = ["什","陌","参","陆","幺"]
    number_list = ["一","二","两","三","四","五","六","七","八","九","零","十","百","千","万","点","加","减","乘","除","根","号"] # ,"几","多少","等"
    for i in range(0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
        
        if (text_1 + "\t" + text_2) not in misspelling_list:
            # 检查是否包含加减乘除
            is_text_1_contain_jzcc = False
            is_text_2_contain_jzcc = False
            index_text_1_jzcc = 0 
            index_text_2_jzcc = 0 
            # 依据cal_word中的计算符号进行检查
            for cal_word in cal_words:
                for x in range (0,3):
                    if cal_word[x] in text_1:
                        is_text_1_contain_jzcc = True
                        index_text_1_jzcc = text_1.index(cal_word[x])
                    if cal_word[x] in text_2:
                        is_text_2_contain_jzcc = True
                        index_text_2_jzcc = text_2.index(cal_word[x])

            # 检查加减乘除符号前方后方是否有数字
            is_text_1_contain_num_before_jzcc = False
            is_text_1_contain_num_after_jzcc = False
            is_text_2_contain_num_before_jzcc = False
            is_text_2_contain_num_after_jzcc = False
            for y in range (0,index_text_1_jzcc):
                if text_1[y].isnumeric() and text_1[y] not in exclusion_words:
                    is_text_1_contain_num_before_jzcc = True
            for y in range (index_text_1_jzcc, len(text_1)):
                if text_1[y].isnumeric() and text_1[y] not in exclusion_words:    
                    is_text_1_contain_num_after_jzcc = True 
            for y in range (0,index_text_2_jzcc):
                if text_2[y].isnumeric() and text_2[y] not in exclusion_words:
                    is_text_2_contain_num_before_jzcc = True
            for y in range (index_text_2_jzcc, len(text_2)):
                if text_2[y].isnumeric() and text_2[y] not in exclusion_words:    
                    is_text_2_contain_num_after_jzcc = True 
            
            # 若包含加减乘除且前后皆有数字
            if is_text_1_contain_jzcc and is_text_2_contain_jzcc and is_text_1_contain_num_before_jzcc and is_text_1_contain_num_after_jzcc and is_text_2_contain_num_before_jzcc and is_text_2_contain_num_after_jzcc:
                # 将符号和全称 替换为缩写
                for cal_word in cal_words:
                    text_1 = text_1.replace(cal_word[2],cal_word[1]) 
                    text_1 = text_1.replace(cal_word[0],cal_word[1])
                    text_2 = text_2.replace(cal_word[2],cal_word[1]) 
                    text_2 = text_2.replace(cal_word[0],cal_word[1])
                # 替换 . 和 =
                for z in range (1, len(text_1)-1):
                    # 只替换前后有数字的“点”
                    if text_1[z-1].isnumeric() and text_1[z] == "." and text_1[z+1].isnumeric():
                        text_1 = text_1[:z] + "点" + text_1[z+1:]
                    # 其余变成“ ”
                    elif text_1[z] == "." :
                        text_1 = text_1[:z] + " " + text_1[z+1:]
                for z in range (1, len(text_2)-1):
                    # 只替换前后有数字的“点”
                    if text_2[z-1].isnumeric() and text_2[z] == "." and text_2[z+1].isnumeric():
                        text_2 = text_2[:z] + "点" + text_2[z+1:]
                    # 其余变成“ ”
                    elif text_2[z] == "." :
                        text_2 = text_2[:z] + " " + text_2[z+1:]
                # 替换 等号
                text_1 = text_1.replace("等于","等") 
                text_2 = text_2.replace("等于","等") 
                text_1 = text_1.replace("=","等") 
                text_2 = text_2.replace("=","等") 
                
                # 找出所有数字
                nums_in_text_1 = re.findall(r'\d+', text_1)
                nums_in_text_2 = re.findall(r'\d+', text_2)
                
                # 若找到数字，则将阿拉伯数字为汉字
                if nums_in_text_1!=[]:
                    for num in nums_in_text_1:
                        word_num_1 = replace_number_with_word(num)
                        text_1 = text_1.replace(num,word_num_1) 
                if nums_in_text_2!=[]:
                    for num in nums_in_text_2:
                        word_num_2 = replace_number_with_word(num)
                        text_2 = text_2.replace(num,word_num_2) 
                
                # 将“ ” 替换为 “”
                text_1 = text_1.replace(" ","")
                text_2 = text_2.replace(" ","")
                

                # 如果经过上述操作后，完全相等的句子着标签为1
                if text_1 == text_2:
                    # print("完全相等: " + text_1 + "\t" + text_2)
                    cal_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                    if data_label_list!= None:
                        cal_1_label_list.append(str(i) + "\t" + data_label_list[i]) 
                # 若句子不相等
                else:
                    #首先观察是否包含不同的加减乘除符号，若有则标记为0
                    has_different_jzcc = False
                    for cal_word in cal_words: 
                        if (cal_word[1] in text_1 and cal_word[1] not in text_2) or (cal_word[1] in text_2 and cal_word[1] not in text_1):
                            cal_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                cal_0_label_list.append(str(i) + "\t" + data_label_list[i])
                            has_different_jzcc = True
                            break
                    # 然后若包含相同的加减乘除符号
                    if not has_different_jzcc:
                        has_diff_num_cal_word = False
                        # 检查加减乘除的数量是否一致
                        for cal_word in cal_words:
                            count_cal_word_1 = text_1.count(cal_word[1])
                            count_cal_word_2 = text_2.count(cal_word[1])
                            # 若数量不一致
                            if count_cal_word_1 != count_cal_word_2:
                                has_diff_num_cal_word = True
                                cal_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                                if data_label_list!= None:
                                    cal_0_label_list.append(str(i) + "\t" + data_label_list[i])
                                has_different_jzcc = True
                                break
                        # 若数量一致
                        if not has_diff_num_cal_word:
                            #尝试将连续的数字与符号之外的内容去掉
                            text_1_new = ""
                            text_1_diff = ""
                            text_2_new = ""
                            text_2_diff = ""
                            for z in range(0, len(text_1)):
                                if text_1[z] in number_list:
                                    text_1_new = text_1_new + text_1[z]
                                else: 
                                    text_1_diff = text_1_diff + text_1[z]
                            for z in range(0, len(text_2)):
                                if text_2[z] in number_list:
                                    text_2_new = text_2_new + text_2[z]
                                else: 
                                    text_2_diff = text_2_diff + text_2[z]
                            # print(text_1_new, text_1_diff)
                            # print(text_2_new, text_2_diff)
                            is_contain_o_h_m = False
                            operator_how_many = ["乘几","加几","减几","除几", "乘多少","加多少","减多少","除多少"]
                            for ohm in operator_how_many:
                                if (ohm in text_1 and ohm not in text_2) or (ohm in text_2 and ohm not in text_1):
                                    is_contain_o_h_m = True
                            # 若计算内容不一致
                            if text_1_new != text_2_new and not is_contain_o_h_m:
                                is_switch_time_add = False
                                # 若包含加法和乘法，则需要判断计算符号前后数字是否是对调的
                                operators_1 = ["乘","加"]
                                for operator in operators_1:
                                    if (operator in text_1_new and operator in text_2_new):
                                        if check_time_add(text_1_new, text_2_new, operator):
                                            is_switch_time_add = True
                                            cal_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i])
                                            if data_label_list!= None:
                                                cal_1_label_list.append(str(i) + "\t" + data_label_list[i])
                                # 若不是对调的，且饱含 除法 和 减法
                                if not is_switch_time_add:
                                    cal_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i])
                                    if data_label_list!= None:
                                        cal_0_label_list.append(str(i) + "\t" + data_label_list[i])
                            else: 
                                # 如果一句包含另一句，且计算相等
                                if text_1 in text_2 or text_2 in text_1:
                                    abc = 1
                                    # cal_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i])
                                    # if data_label_list!= None:
                                    #     cal_1_label_list.append(str(i) + "\t" + data_label_list[i])
                                else:
                                    is_contain_keywords = False
                                    if not is_contain_o_h_m:
                                        # 如果一句饱含keyword 且计算相等
                                        key_words = ["多少","算","等","做","几","语言"]
                                        for key in key_words:
                                            if key in text_1 or key in text_2:
                                                is_contain_keywords = True
                                                cal_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i])
                                                if data_label_list!= None:
                                                    cal_1_label_list.append(str(i) + "\t" + data_label_list[i])
                                    # 如果不包含keyword
                                    if not is_contain_keywords:
                                        count = count + 1
                                        # print("Other : " + text_1 + "\t" + text_2 + "\t" + data_label_list[i])
                                        cal_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i])
                                        if data_label_list!= None:
                                            cal_0_label_list.append(str(i) + "\t" + data_label_list[i])

        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for calculations identification have processed " %((i/len(data_list_1))*100))    
    if print_progress: print("="*15 + "     Done    " + "="*15)
    return cal_1_text_pairs_list, cal_1_label_list, cal_0_text_pairs_list, cal_0_label_list

In [ ]:
# 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
cal_1_text_pairs_list, cal_1_label_list, cal_0_text_pairs_list, cal_0_label_list = get_textpairs_with_calculations(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print("cal_1_text_pairs_list: " + str(len(cal_1_text_pairs_list)))
print("cal_0_text_pairs_list: " + str(len(cal_0_text_pairs_list)))
write_file("common_data/num_1_by_word.txt",cal_1_text_pairs_list)
write_file("common_data/num_0_by_word.txt",cal_0_text_pairs_list)

# 打印结果
len(cal_1_text_pairs_list)
print_dataset(cal_1_text_pairs_list, cal_1_label_list,50)
len(cal_0_text_pairs_list)
print_dataset(cal_0_text_pairs_list, cal_0_label_list,50)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000

=============== In Progress ===============
0.00 of text pairs for calculations identification have processed 
10.00 of text pairs for calculations identification have processed 
20.00 of text pairs for calculations identification have processed 
30.00 of text pairs for calculations identification have processed 
40.00 of text pairs for calculations identification have processed 
50.00 of text pairs for calculations identification have processed 
60.00 of text pairs for calculations identification have processed 
70.00 of text pairs for calculations identification have processed 
80.00 of text pairs for calculations identification have processed 
90.00 of text pairs for calculations identification have processed 
===============     Done    ===============
cal_1_text_pairs_list: 148
cal_0_text_pairs_list: 1837
===============     统

## 4.5 对Active/Passive的句子对的处理

In [ ]:
# 从前往后或从后往前去掉所有相同的字
def remove_same_text(text_1,text_2):
    # 首先，从后往前去掉相同的自     
    if text_1[-1] == text_2[-1]:
        i = 0
        while i < len(text_1) and i < len(text_2):
            x = len(text_1) - 1 - i
            y = len(text_2) - 1 - i
            if text_1[x] == text_2[y]:
                text_1 = text_1[:x] + "嫑" + text_1[x+1:]
                text_2 = text_2[:y] + "嫑" + text_2[y+1:]
                i = i + 1
            else:
                break
    # 将所有发现的相同字替换为
    text_1 = text_1.replace("嫑","")
    text_2 = text_2.replace("嫑","")

    word_index_1 = 0
    word_index_2 = 0
    word_1 = ""
    word_2 = ""
    # 然后，处理 word前后相同的字
    if "把" in text_1:
        word_1 = "把"
        word_index_1 = text_1.index("把")
    elif "被" in text_1:
        word_1 = "被"
        word_index_1 = text_1.index("被")
    if "把" in text_2:
        word_2 = "把"
        word_index_2 = text_2.index("把")
    elif "被" in text_2:
        word_2 = "被"
        word_index_2 = text_2.index("被")
    
    a1 = text_1[0:word_index_1]
    a2 = text_1[word_index_1+1:len(text_1)]
    b1 = text_2[0:word_index_2]
    b2 = text_2[word_index_2+1:len(text_2)]
    # print(a1,a2)
    # print(b1,b2)

    for i in range(0, len(a1)):
        if a1[i] not in b2:
            a1 = a1[:i] + "嫑" + a1[i+1:]
    for i in range(0, len(b1)):
        if b1[i] not in a2:
            b1 = b1[:i] + "嫑" + b1[i+1:]
    for i in range(0, len(a2)):
        if a2[i] not in b1:
            a2 = a2[:i] + "嫑" + a2[i+1:]
    for i in range(0, len(b2)):
        if b2[i] not in a1:
            b2 = b2[:i] + "嫑" + b2[i+1:]
    text_1 = a1 + word_1 + a2
    text_2 = b1 + word_2 + b2

    text_1 = text_1.replace("嫑","")
    text_2 = text_2.replace("嫑","")

    word_to_removed = ["的"]
    for wtr in word_to_removed:
        text_1 = text_1.replace(wtr,"")
        text_ = text_1.replace(wtr,"")

    return text_1, text_2


# 判断是否是 A把B = B被A 的情况，对比 把 和 被 前后的 词是否可以对换
def is_active_passive_bei_ba(text_1,text_2):
    ba_bei = ["把","被"]
    a_1 = ""
    a_2 = ""
    b_1 = ""
    b_2 = ""
    for x in ba_bei:
        if x in text_1:
            ba_index_text_1 = text_1.index(x)
            a_1 = text_1[0:ba_index_text_1]
            a_2 = text_1[ba_index_text_1 + 1: len(text_1)]
        if x in text_2:
            bei_index_text_2 = text_2.index(x)
            b_1 = text_2[0:bei_index_text_2]
            b_2 = text_2[bei_index_text_2 + 1: len(text_2)]
    # print("a_1: " + a_1)
    # print("a_2: " + a_2)
    # print("b_1: " + b_1)
    # print("b_2: " + b_2)
    if (a_1 != "" or b_1 != "") and a_1 == b_2 and a_2 == b_1: 
        # print(True)
        return True
    else:
        # print(False) 
        return False

# 如 A被BC = BCA. 如：武松被谁设计陷害 谁设计陷害武松
def is_active_passive_bei(text_1,text_2):
    a_1 = ""
    a_2 = ""
    if "被" in text_1:
        ba_index_text_1 = text_1.index("被")
        a_1 = text_1[0:ba_index_text_1]
        a_2 = text_1[ba_index_text_1 + 1: len(text_1)]
        text_temp = a_2 + a_1
        if text_temp == text_2 and text_temp!= "":
            return True
    elif "被" in text_2:
        ba_index_text_2 = text_2.index("被")
        a_1 = text_2[0:ba_index_text_2]
        a_2 = text_2[ba_index_text_2 + 1: len(text_2)]
        text_temp = a_2 + a_1
        if text_temp == text_1 and text_temp!= "":
            return True
    return False

In [ ]:
from pypinyin import lazy_pinyin
# 方法3-6 抽取包含“被","把"的句子对
def get_active_passive_by_word(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    ap_1_text_pairs_list = []
    ap_1_label_list = []
    ap_0_text_pairs_list = []
    ap_0_label_list = []
    text_pairs_list = []
    label_list = []
    c_1 = 0
    c_2 = 0
    c_3 = 0
    c_4 = 0
    if print_progress: print("\n" + "="*15 + " In Progress " + "="*15)
    count = 0
    for i in range(0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
        # 已有的规则
        if (text_1 + "\t" + text_2) not in misspelling_list:
            #  "被" 存在于两个句子
            if "被" in text_1 or "被" in text_2:
                subset_1, subset_2 = check_subset(text_1, text_2)
                # 两句差别为同音字则 1
                if subset_1 != "Not_The_Case" and subset_1 != "" and subset_2 != "" and lazy_pinyin(subset_1) == lazy_pinyin(subset_2):
                    c_4 = c_4 + 1
                    ap_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                    if data_label_list!= None:
                        ap_1_label_list.append(str(i) + "\t" + data_label_list[i]) 
                # 情况1 (c_1)，两句都有 “被”
                elif "被" in text_1 and "被" in text_2 : #让/叫 
                    c_1 = c_1 + 1
                    subset_1, subset_2 = check_subset(text_1, text_2)
                    # 若一句包含另一句，且不同部分为不 则 为 0 否则为 1
                    if (subset_1 == "不" and subset_2 == "") or (subset_2 == "不" and subset_1 ==""):
                        ap_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                        if data_label_list!= None:
                            ap_0_label_list.append(str(i) + "\t" + data_label_list[i]) 
                # 情况2（c_2）“把” 和 “被” 同时存在于两个句子中
                elif ("被" in text_1 and "把" in text_2) or ("被" in text_2 and "把" in text_1):
                    # A把B = B被A 先找到A和B
                    c_2 = c_2 + 1
                    text_1_temp, text_2_temp = remove_same_text(text_1,text_2)
                    is_similar = is_active_passive_bei_ba(text_1_temp, text_2_temp)
                    # 若为 A把B / B被A 则为 1
                    if is_similar:
                        ap_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                        if data_label_list!= None:
                            ap_1_label_list.append(str(i) + "\t" + data_label_list[i]) 
                else:
                    if is_active_passive_bei(text_1,text_2):
                        ap_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                        if data_label_list!= None:
                            ap_1_label_list.append(str(i) + "\t" + data_label_list[i])
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for active/passive identification have processed " %((i/len(data_list_1))*100))    
    print("c_1 = " + str(c_1))
    print("c_2 = " + str(c_2))
    print("c_3 = " + str(c_3))
    print("c_4 = " + str(c_4))
    if print_progress: print("="*15 + "     Done    " + "="*15)
    print("active / passive 相关的句子对有: " + str(count) + " 对\n")
    return ap_1_text_pairs_list, ap_1_label_list, ap_0_text_pairs_list, ap_0_label_list

In [ ]:
# # 准备句子对和标签
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")

# # 寻找包含指定字或字符的句子对
ap_1_text_pairs_list, ap_1_label_list, ap_0_text_pairs_list, ap_0_label_list = get_active_passive_by_word(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print("ap_1_text_pairs_list: " + str(len(ap_1_text_pairs_list)))
print("ap_0_text_pairs_list: " + str(len(ap_0_text_pairs_list)))
# text_pairs_list, label_list = get_active_passive(test_text_pair_1_list,test_text_pair_2_list, label_list, True)
write_file("common_data/ap_1_by_word.txt",ap_1_text_pairs_list)
write_file("common_data/ap_0_by_word.txt",ap_0_text_pairs_list)

# # 打印结果
print_dataset(ap_1_text_pairs_list, ap_1_label_list,20)
print_dataset(ap_0_text_pairs_list, ap_0_label_list,20)



The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000

=============== In Progress ===============
0.00 of text pairs for active/passive identification have processed 
10.00 of text pairs for active/passive identification have processed 
20.00 of text pairs for active/passive identification have processed 
30.00 of text pairs for active/passive identification have processed 
40.00 of text pairs for active/passive identification have processed 
50.00 of text pairs for active/passive identification have processed 
60.00 of text pairs for active/passive identification have processed 
70.00 of text pairs for active/passive identification have processed 
80.00 of text pairs for active/passive identification have processed 
90.00 of text pairs for active/passive identification have processed 
c_1 = 225
c_2 = 29
c_3 = 0
c_4 = 0
===============     Done    ===============
active / passive 相关的句

## 4.6 校正Syntatic Structure预测值

In [ ]:
# test a text pairs and predict labels
test_a_data_line_list = read_candidates_line("work/test_B_processed.txt")
print("test_a_data_line_list: " + str(len(test_a_data_line_list)))

predict_label_list = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")
print("predict_label_list: " + str(len(predict_label_list)))

# symmetry 读取句子对
symmetry_list = read_candidates_line("common_data/symmetry_1_by_word.txt")
asymmetry_0_list = read_candidates_line("common_data/asymmetry_0_by_word.txt")
asymmetry_1_list = read_candidates_line("common_data/asymmetry_1_by_word.txt")
negative_symmetry_0_list = read_candidates_line("common_data/negative_asymmetry_0_by_word.txt")
negative_symmetry_1_list = read_candidates_line("common_data/negative_asymmetry_1_by_word.txt")
num_0_list = read_candidates_line("common_data/num_0_by_word.txt")
num_1_list = read_candidates_line("common_data/num_1_by_word.txt")
ap_0_list = read_candidates_line("common_data/ap_0_by_word.txt")
ap_1_list = read_candidates_line("common_data/ap_1_by_word.txt")
antonym_0_list = read_candidates_line("common_data/antonym_0_text_pairs.txt")

to_be_0_list = asymmetry_0_list + negative_symmetry_0_list + num_0_list + ap_0_list + antonym_0_list
to_be_1_list = symmetry_list + asymmetry_1_list + negative_symmetry_1_list + num_1_list + ap_1_list

syntatic_structure = to_be_0_list + to_be_1_list

print("identify " + str(len(syntatic_structure)) + " pairs of syntatic structure")
write_file("common_data/syntatic_structure_list.txt", list(set(syntatic_structure)))

# 赋值
adjusted_predict_0_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_0_list:
        adjusted_predict_0_list_index_label_list.append(str(i)+"\t0")


adjusted_predict_1_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_1_list:
        adjusted_predict_1_list_index_label_list.append(str(i)+"\t1")

adjusted_predict_list_index_label_list = adjusted_predict_0_list_index_label_list + adjusted_predict_1_list_index_label_list

# 使用 方法4-1 来 校正 预测值 
final_predict_lables = adjust_predict_label(test_a_data_line_list, predict_label_list, adjusted_predict_list_index_label_list, True)

# 写入文件
write_file("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv",final_predict_lables)

test_a_data_line_list: 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
predict_label_list: 100000
identify 3218 pairs of syntatic structure
test_a_text_pairs: 100000
final_predict_lables100000
index_and_adjusted_labels3170
十五乘十怎么算	十五乘十五等
index:1129
predict_label:1
adjusted_label:0
六百加六百是稍加	六百加六百相等几百
index:2458
predict_label:1
adjusted_label:0
三个月宝宝睡觉老是摇头怎么回事	三个月宝宝睡觉偶尔摇头怎么回事
index:2806
predict_label:1
adjusted_label:0
二加六加六	六加二加几
index:3234
predict_label:1
adjusted_label:0
九百一十除十四除小	九百一十除十四等多多少少
index:3920
predict_label:1
adjusted_label:0
五十六乘九十九加五十六的简便运算	五十六加九十九乘五十六的简便运算
index:4778
predict_label:1
adjusted_label:0
不生气的歌	不高兴的歌
index:5246
predict_label:1
adjusted_label:0
睡觉老是做梦是什么原因	睡觉偶尔做梦是什么原因
index:5673
predict_label:1
adjusted_label:0
睡觉老是手麻	睡觉偶尔手麻
index:6045
predict_label:1
adjusted_label:0
金鱼凹眼什么原因	金鱼凸眼什么原因
index:7634
predict_label:1
adjusted_label:0
三加五等多少	三加五加几等多少
index:7687
predict_label:1
adjusted_label:0
老是干呕想吐是怎么回事	偶尔干呕想吐是

## 4.7 对比校正结果

In [ ]:
#读取test和两个预测值
# test_text_list_1, test_text_list_2 = read_candidates_test("test_B_1118.tsv")
test_text_list_1, test_text_list_2 = read_candidates_test("work/test_B_processed.txt")
test_predict_label_list_1 = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")
test_predict_label_list_2 = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")
print(len(test_text_list_1))
print(len(test_text_list_2))
print(len(test_predict_label_list_1))
print(len(test_predict_label_list_2))

##### 对比两组预测值，并输出不同的预测结果
def compre_predict_results(text_list1, text_list2, candidates_1, candidates_2, print_correction = False):
    list1 = []
    list2 = []
    label1 = []
    label2 = []
    j = 0
    for i in range (0,len(candidates_1)): 
        if candidates_1[i] != candidates_2[i]:
            if print_correction: print("No. " + str(i) + " label 1 = " + str(candidates_1[i]) + " lable 2 = " + str(candidates_2[i]) + " text pair: " + text_list1[i] + " " + text_list2[i])
            list1.append(text_list1[i])
            list2.append(text_list2[i])
            label1.append(candidates_1[i])
            label2.append(candidates_2[i])
            j = j + 1
    print("Total number of differences is " + str(j))
    print("Two prediction's similarity is %.2f"%((len(candidates_2) - j)/len(candidates_2)*100))
    return list1, list2, label1, label2

text_list1, text_list2, correct_text_list1, predict_text_list1= compre_predict_results(test_text_list_1, test_text_list_2, test_predict_label_list_1, test_predict_label_list_2, True)
print("不同的预测值共有： " + str(len(predict_text_list1)) + " 组")


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000

The number of predict result lines in predict_results/ms_ss_ccf_qianyan_qm_result_B.csv is 100000
100000
100000
100000
100000
No. 1129 label 1 = 1 lable 2 = 0 text pair: 十五乘十怎么算 十五乘十五等
No. 1265 label 1 = 0 lable 2 = 1 text pair: 把游戏空间删了怎么办 游戏空间被不小心删除了怎么找回
No. 2458 label 1 = 1 lable 2 = 0 text pair: 六百加六百是稍加 六百加六百相等几百
No. 2781 label 1 = 0 lable 2 = 1 text pair: 北大哲学系比清华好是吗 清华哲学系比北大的差些是吗
No. 2806 label 1 = 1 lable 2 = 0 text pair: 三个月宝宝睡觉老是摇头怎么回事 三个月宝宝睡觉偶尔摇头怎么回事
No. 3234 label 1 = 1 lable 2 = 0 text pair: 二加六加六 六加二加几
No. 3662 label 1 = 0 lable 2 = 1 text pair: 西安到北京高铁多长时间 北京到西安高铁多长时间
No. 3920 label 1 = 1 lable 2 = 0 text pair: 九百一十除十四除小 九百一十除十四等多多少少
No. 4778 label 1 = 1 lable 2 = 0 text pair: 五十六乘九十九加五十六的简便运算 五十六加九十九乘五十六的简便运算
No. 5078 label 1 = 0 lable 2 = 1 text pair: 广州到武汉的飞机票多少钱 武汉到广州的飞机票多少钱
No. 5246 label 1 = 1 lable 2 = 0 text p

## 5.1 Insert N/V/ADJ/ADV...

In [ ]:
# 找到 Word & Phrase - insert n/v/adj/adv...
def identify_insert_word_phrase(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling和syntactic structure处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    syntactic_structure_list = read_candidates_line("common_data/syntatic_structure_list.txt")
    # replace_0_list = read_candidates_line("replace_0_by_word.txt")
    # replace_1_list = read_candidates_line("replace_1_by_word.txt")
    # 没有特别重要意义的词语
    meanlingless_word_list = ["小布","小爱","小艾","小步","小度","小o","小v","小薇","小冰","app"]
    # [by][Language][How]
    critical_words_list = []
    critibal_connection = ["用","的",""]
    critical_language =["汉语","中文","英语","英文","韩语","韩文","日语","日文","德语","德文","拼音","作文","词组","笔顺","笔画","读音","电话号码"]
    critical_how =["怎么拼","怎拼","怎么说","怎说","怎么读","怎读","怎么写","怎写","是什么",""]      
    for connection in critibal_connection:
        for language in critical_language:
            for how in critical_how:
                critical_words_list.append(connection+language+how)
    critical_words_list = critical_words_list + ["等于多少克","等于多少千克","等于多少斤","等于多少公斤","是什么意思","什么意思","十么意思","怎么拼音"]
    # 确认能够提高的list
    insert_loc_per_org_list = ["LOC","PER","ORG"]
    insert_negate_list = ["不","没","无","莫","非","勿"] # 可与尝试 "是不是" "不是",
    insert_temporal_list = ["t","TIME"] #
    # 不确定是否能够提高的list  存疑，太复杂 
    insert_v_list = ["vd","vn"] # "v" 太多,"vd" 无
    insert_adj_list = ["ad", "an"] #
    insert_n_list = ["n"] #,"n"
    insert_adv_list = ["d"] #
    insert_n_other_list = ["nr","ns","nt","nw","nz"] #
    # 存放修正后的句子对
    insert_1_list = []
    label_1_list = []
    insert_0_list = []
    label_0_list = []
    if print_progress: print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
        
        text_1 = text_1.replace(" ","")
        text_2 = text_2.replace(" ","")
        
        # 获取两句话的不同部分
        subset_1, subset_2 = check_subset(text_1,text_2)
        
        text_pair = text_1 + "\t" + text_2

        if text_pair not in (misspelling_list + syntactic_structure_list):# + replace_0_list + replace_1_list): 
            # 若一致则为1
            if text_1 == text_2:
                abc = 1
                insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
            # 若存在一段不一致的字符串
            elif subset_1!="Not_The_Case" and subset_2!= "Not_The_Case":
                # 找出所有insert的句子对，获取被插入的字符
                insert_words_original = ""
                if subset_1!= "" and subset_2 == "":
                    insert_words_original = subset_1
                elif subset_1=="" and subset_2 !="":
                    insert_words_original = subset_2

                # 当插入的字符大于等于1个字符时
                if len(insert_words_original) >= 1:
                    #分析该插入字符的词性
                    words_insert, nature_insert = get_words_and_words_nature(insert_words_original)
                    
                    # 情况1 判断插入是否是 数字 或者 字母
                    is_insert_contain_only_numeric = True
                    is_insert_contain_only_alpha = True
                    
                    for ni in insert_words_original:
                        # 有些大写字和一容易混淆，排除
                        if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                            is_insert_contain_only_numeric = False
                        if not ni.encode('UTF-8').isalpha():
                            is_insert_contain_only_alpha = False
                    
                    # 情况 2 句子对带有需要严格区分内容的
                    # 判断是否包含严格语句
                    is_critical_case_0 = False
                    is_critical_case_1 = False
                    
                    for c_i in range(0, len(critical_words_list)):
                        if critical_words_list[c_i] != "" and critical_words_list[c_i] in text_1 and critical_words_list[c_i] in text_2:

                            # 去掉这些严格词语
                            a = text_1.rstrip(critical_words_list[c_i])
                            b = text_2.rstrip(critical_words_list[c_i])

                            # 剩下的词语 + add words 
                            add_words = ["的","的用","这用","用","这是","是","字"]
                            is_a_or_b_with_add_words = False
                            for a_w in add_words:
                                if (a not in add_words and b not in add_words and a != "" and b!= "") and (a == b or a == b + a_w or a + a_w== b):
                                    is_a_or_b_with_add_words = True
                            # 如果剩下的相等则为1
                            if is_a_or_b_with_add_words:
                                is_critical_case_1 = True
                                break
                            # 如果剩下的不等则为0
                            else:
                                is_critical_case_0 = True
                                break
            
                    # 情况 3 同一个内容在一个句子中给重复出现多次
                    # 判断 text_1 和 text_2 是否有加减乘除
                    is_contain_jzcc = False
                    for jzcc_word in ["加","减","乘","除"]:
                        if jzcc_word in text_1 or jzcc_word in text_2:
                            is_contain_jzcc = True
                    # 判断 text_1 和 text_2 是否有数字
                    is_contain_number = False
                    for number_word in ["一","二","三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]:
                        if number_word in text_1 or number_word in text_2:
                            is_contain_number = True
                    # 判断 text_1 和 text_2 是否都是汉字
                    is_text_1_all_chinese = True
                    is_text_2_all_chinese = True
                    for ch in text_1:
                        if not ('\u4e00' <= ch <= '\u9fa5'):
                            is_text_1_all_chinese = False
                    for ch in text_2:
                        if not ('\u4e00' <= ch <= '\u9fa5'):
                            is_text_2_all_chinese = False
                    # 判断是否重复内容
                    is_repeat_content = False
                    elements_in_text_1 = list(set(text_1))
                    elements_in_text_2 = list(set(text_2))
                    new_text_1 = ""
                    new_text_2 = ""
                    for element in elements_in_text_1:
                        new_text_1 = new_text_1 + element
                    for element in elements_in_text_2:
                        new_text_2 = new_text_2 + element
                    if new_text_1 == new_text_2 and new_text_1 in text_2 and new_text_2 in text_1 and not (is_contain_jzcc and is_contain_number) :
                        is_repeat_content = True

                    # 情况4 鲁棒性增强句子对的判断
                    is_text_1_contain_number = False
                    is_text_2_contain_number = False
                    for number_word in ["三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]:
                        if number_word in text_1:
                            is_text_1_contain_number = True
                        if number_word in text_2:
                            is_text_2_contain_number = True
                    
                    # 如果插入的字词已经在两个句子中 (Verified train)
                    if insert_words_original in meanlingless_word_list:
                        abc = 1
                        insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # 仅插入这些字符
                    elif len(insert_words_original) == 1 and ((insert_words_original in ["三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]) and (not (is_text_1_contain_number and is_text_2_contain_number))) or is_insert_contain_only_alpha:
                        abc = 1
                        insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # Insert Negate 如果包含 否定词，且 否定词 不在最后一个字 (verfified train/test)
                    elif insert_words_original in insert_negate_list:
                        abc = 1
                        # 如果否定词在最后可以表示问句
                        if  text_1[-1] == "不" or text_2[-1] == "不" or "不我" in text_1 or "不我" in text_2 or "不你" in text_1 or "不你" in text_2 or "不帮我" in text_1 or "不帮我" in text_2 or "不小布" in text_1 or "不小布" in text_2:
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # Insert temporal (verfified train)
                    elif len(words_insert) == 1 and nature_insert[0] in insert_temporal_list:
                        is_current_time = insert_words_original in ["现在","今天","今日","早上","目前","当前","此时","平时","最近","当前","今年","每日","每天","近期","此刻","唐","宋","元","明","清"]
                        # 是否是需要严格区分的情况 如用英语怎么说
                        if is_critical_case_0 and is_current_time:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                        # 一种是插入了默认的时间点，另一种是同一个时间点的重复（当然要排除数学计算上的误判，如二点五加二点五）
                        elif is_current_time or ((not insert_words_original[0].isnumeric()) and (insert_words_original in text_1 and insert_words_original in text_2)) :
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                        # 其余时间信息则为 0
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i]))   
                    # Insert LOC/PER/ORG (verfified train)(混乱)
                    elif len(words_insert) == 1 and nature_insert[0] in insert_loc_per_org_list:
                        abc = 1 
                        if (insert_words_original in text_1 and insert_words_original in text_2) or (insert_words_original in ["省","市","镇","乡","县","村","岛","区"]):
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                    # # Insert v  (混乱)
                    # elif len(insert_words_original) >= 2 and len(words_insert) == 1 and nature_insert[0] in insert_v_list:
                    #     abc = 1
                    #     # if (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # # Insert adj  (混乱)
                    # elif len(words_insert) == 1 and nature_insert[0] in insert_adj_list:
                    #     abc = 1
                    #     # insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     # if data_label_list!=None:
                    #     #     label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # # Insert adv  (混乱)
                    # elif len(insert_words_original) >= 1 and len(words_insert) == 1 and nature_insert[0] in insert_adv_list:
                    #     abc = 1 
                    #     # if (insert_words_original in ["到底","很","确实","其实","非常","就","都","究竟","马上","一起","的确","也","已经","居然","赶快","太","一共", "又", "再", "难道", "当然", "实在", "真就", "大概", "赶紧", "简直","立刻", "难怪", "老是", "从来", "才", "好好", "大约", "好像", "先", "总是", "永远"]) or (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # elif (insert_words_original in ["一直"]) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    # # Insert n (混乱)
                    # elif len(insert_words_original) >= 2 and len(words_insert) == 1 and nature_insert[0] in insert_n_list:
                    #     abc = 1  #手机
                    #     # if (insert_words_original in ["天下","意思","","","","",""]) or (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                
    
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for insert subset identification have processed " %((i/len(data_list_1))*100))    
    if print_progress: print("="*15 + "     Done    " + "="*15)
    print("插入 指定 natures 的句子对共 " + str(len(insert_1_list) + len(insert_0_list)) + " 组\n")
    return insert_1_list, label_1_list, insert_0_list, label_0_list

In [ ]:
# 寻找Insert的句子对
# 准备句子对和标签
test_text_pair_1_list, test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
insert_1_list, label_1_list, insert_0_list, label_0_list = identify_insert_word_phrase(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/insert_1_by_word.txt",insert_1_list)
write_file("common_data/insert_0_by_word.txt",insert_0_list)

# 打印结果
print_dataset(insert_1_list, label_1_list,300)
print_dataset(insert_0_list, label_0_list,300)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ss_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for insert subset identification have processed 
10.00 of text pairs for insert subset identification have processed 
20.00 of text pairs for insert subset identification have processed 
30.00 of text pairs for insert subset identification have processed 
40.00 of text pairs for insert subset identification have processed 
50.00 of text pairs for insert subset identification have processed 
60.00 of text pairs for insert subset identification have processed 
70.00 of text pairs for insert subset identification have processed 
80.00 of text pairs for insert subset identification have processed 
90.00 of text pairs for insert subset identification have processed 
===============     Done    ===============
插入 指定 natures 的句子对共 873 组

===============     统计    ===========

## 5.2 Replace N/V/ADJ/ADV...

In [ ]:
# 找到 Word & Phrase - replace n/v/adj/adv...
def identify_replace_word_phrase(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling和syntactic structure处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    syntactic_structure_list = read_candidates_line("common_data/syntatic_structure_list.txt")
    insert_0_list = read_candidates_line("common_data/insert_0_by_word.txt")
    insert_1_list = read_candidates_line("common_data/insert_1_by_word.txt")
    
    # 载入 同义词 列表
    synonym_list = read_candidates_line("common_data/synonym_list.txt")
    print("="*15 + "synonym_list" + "="*15)
    print(len(synonym_list))
    # [by][Language][How]
    critical_words_list = []
    critibal_connection = ["用","的",""]
    critical_language =["汉语","中文","英语","英文","韩语","韩文","日语","日文","德语","德文","拼音","作文","词组","笔顺","笔画","读音","电话号码","英文翻译","英语翻译","英文单词","英语单词",""]
    critical_how =["怎么拼","怎拼","怎么说","怎说","怎么读","怎读","怎么写","怎写","是什么",""]      
    for connection in critibal_connection:
        for language in critical_language:
            for how in critical_how:
                if connection+language+how not in critibal_connection:
                    critical_words_list.append(connection+language+how)
    critical_words_list = critical_words_list + ["等于多少克","等于多少千克","等于多少斤","等于多少公斤","是什么意思","什么意思","十么意思","怎么拼音"]
    # 各类关注的词性
    # Named Entity
    replace_per_nr_list = ["PER","nr"]
    replace_loc_ns_list = ["LOC","ns"]
    replace_org_nt_list = ["ORG","nt"]
    replace_nw_nz_list = ["nw","nz"]
    # Word/Pharse/Name Entity
    replace_n_list = ["n"] # "n" 
    # Word/Pharse/Synonym
    replace_v_list = ["vd","vn"] #"v","vd" 无,"vn","v",
    # adj/adv/time
    replace_adj_list = ["a", "ad", "an"] # 
    replace_adv_list = ["d"] #
    replace_temporal_list = ["t","TIME"] #

    replace_1_list = []
    label_1_list = []
    replace_0_list = []
    label_0_list = []

    if print_progress: print("="*15 + " In Progress " + "="*15)
    
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
                
        # 若为 已经处理过的情况，则跳过
        if data_list_1[i] + "\t" + data_list_2[i] in (misspelling_list + misspelling_list + insert_0_list + insert_1_list):
            abc = 1
        # 完全一致的为 跳过
        elif text_1 == text_2:
            abc = 1
            replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
            if data_label_list!=None:
                label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
        # 进入replace的处理
        else:  

            # 句子对带有需要严格区分内容的
            # 判断是否包含严格语句
            is_critical_case_0 = False
            is_critical_case_1 = False
                    
            for c_i in range(0, len(critical_words_list)):
                if critical_words_list[c_i] != "" and critical_words_list[c_i] in text_1 and critical_words_list[c_i] in text_2:

                    # 去掉这些严格词语
                    a = text_1.rstrip(critical_words_list[c_i])
                    b = text_2.rstrip(critical_words_list[c_i])
                    # 剩下的词语 + add words 
                    add_words = ["的","的用","这用","用","这是","是","字"]
                    is_a_or_b_with_add_words = False
                    for a_w in add_words:
                        if (a not in add_words and b not in add_words and a != "" and b!= "") and (a == b or a == b + a_w or a + a_w== b):
                            is_a_or_b_with_add_words = True
                    # 如果剩下的相等则为1
                    if is_a_or_b_with_add_words:
                        is_critical_case_1 = True
                        break
                    # 如果剩下的不等则为0
                    else:
                        is_critical_case_0 = True
                        break

            # (1) 获取两句话中被替换的部分 by word 
            subset_1, subset_2 = check_subset(text_1,text_2)
            # 判断插入是否是 字母
            is_replace_contain_only_alpha_1 = True
            is_replace_contain_only_alpha_2 = True
            is_replace_contain_only_numeric_1 = True 
            is_replace_contain_only_numeric_2 = True 

            for ni in subset_1:
                if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                    is_replace_contain_only_numeric_1 = False
                if not ni.encode('UTF-8').isalpha():
                    is_replace_contain_only_alpha_1 = False
            for ni in subset_2:
                if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                    is_replace_contain_only_numeric_2 = False
                if not ni.encode('UTF-8').isalpha():
                    is_replace_contain_only_alpha_2 = False

            s_w_1, s_n_1 = get_words_and_words_nature(subset_1)
            s_w_2, s_n_2 = get_words_and_words_nature(subset_2)

            # (2) 获取两句话中被替换的部分 by nature
            words_1, nature_1 = get_words_and_words_nature(text_1)
            words_2, nature_2 = get_words_and_words_nature(text_2)

            words_natures_1 = merge_word_nature (words_1, nature_1)
            words_natures_2 = merge_word_nature (words_2, nature_2)

            subset_word_nature_1, subset_word_nature_2 = check_subset(words_natures_1,words_natures_2)
            # print( subset_word_nature_1, subset_word_nature_2 )
            # print( is_critical_case_0 )
            # by word
            # 如果替换的是字母
            if subset_1!="Not_The_Case" and subset_2!= "Not_The_Case" and subset_1 != "" and subset_2 != "" and is_replace_contain_only_alpha_1 and is_replace_contain_only_alpha_2:
                abc = 1
                replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # 如果替换的是数字
            elif subset_1!="Not_The_Case" and subset_2!= "Not_The_Case" and subset_1 != "" and subset_2 != "" and is_replace_contain_only_numeric_1 and is_replace_contain_only_numeric_2:
                abc = 1
                replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # 如果替换（by word）对出现在近义词表中
            elif (subset_1!="Not_The_Case") and (subset_2!= "Not_The_Case") and ((subset_1 + "\\t" + subset_2) in synonym_list or (subset_2 + "\\t" + subset_1) in synonym_list) and not is_critical_case_0:
                abc = 1
                replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # by nature
            elif len(subset_word_nature_1) == 1 and len(subset_word_nature_2) == 1 and subset_word_nature_1!="Not_The_Case" and subset_word_nature_2!= "Not_The_Case" and subset_word_nature_1 != "" and subset_word_nature_2 != "" and not is_critical_case_0:    
                t_1 = subset_word_nature_1[0][0] + "\\t" + subset_word_nature_2[0][0] 
                t_2 = subset_word_nature_2[0][0] + "\\t" + subset_word_nature_1[0][0]       
                # 如果 是同义词表中的替换词则为1 
                if t_1 in synonym_list or t_2 in synonym_list:
                    abc =1
                    replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
                # replace per
                elif subset_word_nature_1[0][1] in replace_per_nr_list and subset_word_nature_2[0][1] in replace_per_nr_list:
                    abc = 1
                    replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                # Replace LOC nr
                elif subset_word_nature_1[0][1] in replace_loc_ns_list and subset_word_nature_2[0][1] in replace_loc_ns_list:
                    is_loc_full = False
                    for loc_word in ["国","省","市","镇","乡","县","村","岛","区","景区","林区","地区"]:
                        if  subset_word_nature_1[0][0] == subset_word_nature_2[0][0] + loc_word:
                            is_loc_full = True
                        elif  subset_word_nature_1[0][0]+ loc_word == subset_word_nature_2[0][0] :
                            is_loc_full = True   
                    if is_loc_full:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                # Replace ORG nt
                elif subset_word_nature_1[0][1] in replace_org_nt_list and subset_word_nature_2[0][1] in replace_org_nt_list:
                    is_org_full = False
                    s_1,s_2 = check_subset(subset_word_nature_1[0][0],subset_word_nature_2[0][0])
                    org_words = ["公司","集团","局","部门","处","省","市","镇","乡","县","岛","区"]
                    if s_1 in org_words or s_2 in org_words:
                        is_org_full = True
                    if (s_1+"\\t"+s_2) in synonym_list or (s_2 + "\\t" + s_1) in synonym_list:
                        is_org_full = True
                    if is_org_full:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
            #     # Replace NW/NZ (混乱)
            #     # elif subset_word_nature_1[0][1] in replace_nw_nz_list and subset_word_nature_2[0][1] in replace_nw_nz_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace N  (混乱)
            #     # elif subset_word_nature_1[0][1] in replace_n_list and subset_word_nature_2[0][1] in replace_n_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace V（混乱）
            #     # elif subset_word_nature_1[0][1] in replace_v_list and subset_word_nature_2[0][1] in replace_v_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace adj（混乱）
            #     # elif subset_word_nature_1[0][1] in replace_adj_list and subset_word_nature_2[0][1] in replace_adj_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                # Replace adv
                elif subset_word_nature_1[0][1] in replace_adv_list and subset_word_nature_2[0][1] in replace_adv_list:
                    abc = 1
                    replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                # Replace temporal
                elif subset_word_nature_1[0][1] in replace_temporal_list and subset_word_nature_2[0][1] in replace_temporal_list:
                    is_temp_full = False
                    s_1,s_2 = check_subset(subset_word_nature_1[0][0],subset_word_nature_2[0][0])
                    temp_words = ["月","日","份","个","号","钟"]
                    if s_1 in temp_words or s_2 in temp_words:
                        is_temp_full = True
                    if (s_1+"\\t"+s_2) in synonym_list or (s_2 + "\\t" + s_1) in synonym_list:
                        is_temp_full = True
                    if is_temp_full and not is_critical_case_0:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))                
    
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for replace subset identification have processed " %((i/len(data_list_1))*100))    
    if print_progress: print("="*15 + "     Done    " + "="*15)
    print("替换 指定 natures 的句子对共 " + str(len(replace_1_list) + len(replace_0_list)) + " 组\n")
    return replace_1_list, label_1_list, replace_0_list, label_0_list

In [ ]:
# 寻找Replace的句子对
# 准备句子对和标签
test_text_pair_1_list, test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
replace_1_list, label_1_list, replace_0_list, label_0_list = identify_replace_word_phrase(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/replace_1_by_word.txt",replace_1_list)
write_file("common_data/replace_0_by_word.txt",replace_0_list)

# 打印结果
print_dataset(replace_1_list, label_1_list,50)
print_dataset(replace_0_list, label_0_list,len(replace_0_list))

句子对: 十八字短句	三字短句	86807	0
句子对: 正月十五的习俗急啊	腊月十五的习俗急啊	86812	0
句子对: 张羽茉名字打分	张梦琪名字打分	86813	0
句子对: 苹果六代替home键的小圆点哪里设置	苹果十一代替home键的小圆点哪里设置	86825	0
句子对: 什么是肿瘤四项	什么是肿瘤六项	86852	0
句子对: 战地五内存要求	战地一内存要求	86862	0
句子对: 历史上姓翟的名人有哪些	历史上姓苏的名人有哪些	86889	0
句子对: 八十五除一百	八十五除二	86891	0
句子对: 一百乘二十	五百乘二十	86902	0
句子对: 赣马高级中学校长	潮州高级中学校长	86921	0
句子对: 月经干净后第一天会排卵吗	月经干净后第九天会排卵吗	86942	0
句子对: 阴历三月初四是什么命	阴历四月初四是什么命	86943	0
句子对: 云南警官学院怎么样	云南中医学院怎么样	86950	0
句子对: 中行客服工资	中信客服工资	86967	0
句子对: 四川师范大学校训	广西师范大学校训	86981	0
句子对: 十一年年三十是多少号	十二年年三十是多少号	86994	0
句子对: 大写七字怎么写笔顺	大写一字怎么写笔顺	87002	0
句子对: 山东方言	内蒙古方言	87018	0
句子对: 九百五十一除五的竖式怎么列	零除五的竖式怎么列	87029	0
句子对: 二十乘八	二十乘六	87045	0
句子对: 齐齐哈尔到昂昂溪多少公里	齐齐哈尔到郑州多少公里	87059	0
句子对: 和田到张掖的飞机票	和田到天津的飞机票	87074	0
句子对: 苏州到沈阳火车票价格	北京到沈阳火车票价格	87107	0
句子对: 山西师范大学是双一流吗	陕西师大是双一流吗	87126	0
句子对: 上海交大和北大哪个好	上海交大和人大哪个好	87135	0
句子对: 形容青春的一个字	形容青春的四个字	87157	0
句子对: 三十八年出生男演员	三十九年出生男演员	87164	0
句子对: 三月十一日是什么星座	三月九日是什么星座	87168	0
句子对: 第九的英语怎么说	第二的英语怎么说	87175	0
句子对: 八千开根号等多少	一万三千开根号等多少	87198	0
句子对: 太原到湖州火车时刻表	太原到南昌火车时刻表	872

## 5.3 校正 Lexical Semantic 预测值

In [ ]:
# test a text pairs and predict labels
test_a_data_line_list = read_candidates_line("work/test_B_processed.txt")
print("test_a_data_line_list: " + str(len(test_a_data_line_list)))

predict_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")
print("predict_label_list: " + str(len(predict_label_list)))

# symmetry 读取句子对
insert_0_list = read_candidates_line("common_data/insert_0_by_word.txt")
insert_1_list = read_candidates_line("common_data/insert_1_by_word.txt")
replace_0_list = read_candidates_line("common_data/replace_0_by_word.txt")
replace_1_list = read_candidates_line("common_data/replace_1_by_word.txt")

to_be_1_list = insert_1_list + replace_1_list
to_be_0_list = insert_0_list + replace_0_list

lexical_semantic_list = to_be_0_list + to_be_1_list
print("identify " + str(len(lexical_semantic_list)) + " pairs of lexical semantic")
write_file("common_data/lexical_semantic_list.txt", lexical_semantic_list)

# 赋值
adjusted_predict_0_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_0_list:
        adjusted_predict_0_list_index_label_list.append(str(i)+"\t0")

adjusted_predict_1_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_1_list:
        adjusted_predict_1_list_index_label_list.append(str(i)+"\t1")

adjusted_predict_list_index_label_list = adjusted_predict_0_list_index_label_list + adjusted_predict_1_list_index_label_list

# 使用 方法4-1 来 校正 预测值 
final_predict_lables = adjust_predict_label(test_a_data_line_list, predict_label_list, adjusted_predict_list_index_label_list, True)

# 写入文件
write_file("predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv",final_predict_lables)

predict_label:1
adjusted_label:0
怎样在手机上下载视频	怎样在苹果手机上下载视频
index:46686
predict_label:1
adjusted_label:0
qq群宣传语怎么写	群宣传语怎么写
index:47106
predict_label:1
adjusted_label:0
小米怎么进去recovery模式	小米怎么进去recover模式
index:47336
predict_label:1
adjusted_label:0
阿里巴巴的股价	阿里的股价
index:47816
predict_label:1
adjusted_label:0
开地产公司要多少钱	开房地产公司要多少钱
index:48195
predict_label:1
adjusted_label:0
伯利特股票发行价	伯特利股票发行价
index:48739
predict_label:1
adjusted_label:0
五行属火的吉利字	五行属火的字
index:50388
predict_label:1
adjusted_label:0
石家庄到大寨多少公里	石家庄到山西大寨多少公里
index:50730
predict_label:1
adjusted_label:0
邮政国际邮费价格表	中国邮政国际邮费价格表
index:50929
predict_label:1
adjusted_label:0
彭州二级医院有哪些	彭州二甲医院有哪些
index:51174
predict_label:1
adjusted_label:0
复旦成考录取分数线	上海复旦成考录取分数线
index:51260
predict_label:1
adjusted_label:0
大便一直不正常	大便不不正常
index:51533
predict_label:1
adjusted_label:0
喉咙里好像有异物感	喉咙里一直有异物感
index:51612
predict_label:1
adjusted_label:0
腰围七十六cm等多少尺	腰围七十六等多少尺
index:52119
predict_label:1
adjusted_label:0
广东移动短号怎么取消	移动短号怎么取消
index:52268
predict_label:1


## 5.4 对比校正结果

In [ ]:
#读取test和两个预测值
test_text_list_1, test_text_list_2 = read_candidates_test("work/test_B_1118.tsv")
test_predict_label_list_1 = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")
test_predict_label_list_2 = read_labels("predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv")
print(len(test_text_list_1))
print(len(test_text_list_2))
print(len(test_predict_label_list_1))
print(len(test_predict_label_list_2))

# 对比两组预测值，并输出不同的预测结果
def compre_predict_results(text_list1, text_list2, candidates_1, candidates_2):
    list1 = []
    list2 = []
    label1 = []
    label2 = []
    j = 0
    for i in range (0,len(candidates_1)): 
        if candidates_1[i] != candidates_2[i]:
            print("No. " + str(i) + " label 1 = " + str(candidates_1[i]) + " lable 2 = " + str(candidates_2[i]) + " text pair: " + text_list1[i] + " " + text_list2[i])
            list1.append(text_list1[i])
            list2.append(text_list2[i])
            label1.append(candidates_1[i])
            label2.append(candidates_2[i])
            j = j + 1
    print("Total number of differences is " + str(j))
    print("Two prediction's similarity is %.2f"%((len(candidates_2) - j)/len(candidates_2)*100))
    return list1, list2, label1, label2

text_list1, text_list2, correct_text_list1, predict_text_list1= compre_predict_results(test_text_list_1, test_text_list_2, test_predict_label_list_1, test_predict_label_list_2)
print("不同的预测值共有： " + str(len(predict_text_list1)) + " 组")


The number of lines in work/test_B_1118.tsv is 100000

The number of predict result lines in predict_results/ms_ss_ccf_qianyan_qm_result_B.csv is 100000

The number of predict result lines in predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv is 100000
100000
100000
100000
100000
No. 48 label 1 = 1 lable 2 = 0 text pair: 我是不是真的有点烦人 我是不是真的很烦人
No. 650 label 1 = 1 lable 2 = 0 text pair: 雷克萨斯ls怎么样 雷克萨斯lc怎么样
No. 681 label 1 = 0 lable 2 = 1 text pair: 美国总统是谁 美国总统现在是谁
No. 801 label 1 = 1 lable 2 = 0 text pair: 苹果手机7p屏幕上的悬浮球怎么设置 苹果手机7屏幕上的悬浮球怎么设置
No. 853 label 1 = 1 lable 2 = 0 text pair: 华西医学院院长 华西医院院长
No. 1100 label 1 = 1 lable 2 = 0 text pair: 我为什么不信基督教 我为什么不再信基督教
No. 1530 label 1 = 0 lable 2 = 1 text pair: 退出开车模式 退出车载模式
No. 1885 label 1 = 1 lable 2 = 0 text pair: 北京地坛医院新址在哪 天坛医院新址在哪
No. 2068 label 1 = 1 lable 2 = 0 text pair: 热巴为什么被新浪限流 迪丽热巴为什么被新浪限流
No. 2325 label 1 = 0 lable 2 = 1 text pair: 我可以睡觉吗 我现在可以睡觉吗
No. 2481 label 1 = 1 lable 2 = 0 text pair: 郑州轻工业学院好不好 郑州轻工业大学好不好
No. 2889 label